Harbey Perdomo Montoya  
CC 1072961355

In [1]:
#@title Importar librerias
!pip install PyMuPDF spacy nltk stanza transformers

In [2]:
!python -m spacy download es_core_news_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/es_core_news_sm-3.8.0/es_core_news_sm-3.8.0-py3-none-any.whl (12.9 MB)
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
#@title Importar librerias
import fitz  # PyMuPDF para leer PDF
import unicodedata
import spacy
import nltk
import stanza
from transformers import AutoTokenizer
import re
import textblob

In [4]:
#@title Descargar Modelos
nltk.download('punkt')
stanza.download('es')
nlp_stanza = stanza.Pipeline('es')
nlp_spacy = spacy.load('es_core_news_sm')
tokenizer_bert = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: es (Spanish) ...
INFO:stanza:File exists: /root/stanza_resources/es/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: es (Spanish):
| Processor    | Package           |
------------------------------------
| tokenize     | combined          |
| mwt          | combined          |
| pos          | combined_charlm   |
| lemma        | combined_nocharlm |
| constituency | combined_charlm   |
| depparse     | combined_charlm   |
| sentiment    | tass2020_charlm   |
| ner          | conll02           |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a tok

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
#@title Leer el archivo fuente
# 1. Leer el archivo PDF
def leer_pdf(ruta):
    with fitz.open(ruta) as doc:
        texto = "\n".join([page.get_text("text") for page in doc])
    return texto

ruta_pdf = "/content/drive/MyDrive/Maestria en analítica estratégica de datos/4. Semestrre/Analisis_textos/Taller_2/Sesgos de magnitud.pdf"
texto = leer_pdf(ruta_pdf)
print("Texto original:\n", texto[:500])

Texto original:
 Investigación sobre el sesgo de Magnitud 
TLDR 
Los sesgos de magnitud en estudios sociales se refieren a errores sistemáticos que 
afectan la estimación cuantitativa de efectos, relaciones o parámetros en la 
investigación social, distorsionando tanto la dirección como la magnitud de los 
hallazgos y comprometiendo la validez de las conclusiones. 
Introducción 
Los sesgos de magnitud representan uno de los problemas metodológicos más 
críticos en la investigación social cuantitativa. Estos sesg


In [33]:
# Contar la cantidad de palabras (tokens)
cantidad_palabras = len(texto)
print(f"Cantidad de palabras en el PDF: {cantidad_palabras}")

Cantidad de palabras en el PDF: 21952


In [7]:
# 2. Convertir a minúsculas
texto_minusculas = texto.lower()
print("\nTexto en minúsculas:\n", texto_minusculas[:500])


Texto en minúsculas:
 investigación sobre el sesgo de magnitud 
tldr 
los sesgos de magnitud en estudios sociales se refieren a errores sistemáticos que 
afectan la estimación cuantitativa de efectos, relaciones o parámetros en la 
investigación social, distorsionando tanto la dirección como la magnitud de los 
hallazgos y comprometiendo la validez de las conclusiones. 
introducción 
los sesgos de magnitud representan uno de los problemas metodológicos más 
críticos en la investigación social cuantitativa. estos sesg


In [8]:
# 3. Normalización Unicode (NFC)
def normalizar_unicode(texto):
    return unicodedata.normalize('NFC', texto)

texto_normalizado = normalizar_unicode(texto_minusculas)
print("\nTexto normalizado:\n", texto_normalizado[:500])


Texto normalizado:
 investigación sobre el sesgo de magnitud 
tldr 
los sesgos de magnitud en estudios sociales se refieren a errores sistemáticos que 
afectan la estimación cuantitativa de efectos, relaciones o parámetros en la 
investigación social, distorsionando tanto la dirección como la magnitud de los 
hallazgos y comprometiendo la validez de las conclusiones. 
introducción 
los sesgos de magnitud representan uno de los problemas metodológicos más 
críticos en la investigación social cuantitativa. estos sesg


In [9]:
# Download the necessary data for Spanish tokenization
nltk.download('punkt')
nltk.download('spanish_grammars')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package spanish_grammars to /root/nltk_data...
[nltk_data]   Package spanish_grammars is already up-to-date!


True

In [10]:
# 4. Tokenización con diferentes librerías

def tokenizar_spacy(texto):
    return [token.text for token in nlp_spacy(texto)]

def tokenizar_nltk(texto):
    return nltk.word_tokenize(texto, language='spanish')

def tokenizar_stanza(texto):
    doc = nlp_stanza(texto)
    return [word.text for sent in doc.sentences for word in sent.words]

def tokenizar_bert(texto):
    return tokenizer_bert.tokenize(texto)

In [11]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [12]:
# Aplicamos cada método
tokens_spacy = tokenizar_spacy(texto_normalizado)
tokens_nltk = tokenizar_nltk(texto_normalizado)
tokens_stanza = tokenizar_stanza(texto_normalizado)
tokens_bert = tokenizar_bert(texto_normalizado)

Token indices sequence length is longer than the specified maximum sequence length for this model (5079 > 512). Running this sequence through the model will result in indexing errors


In [13]:
# Mostramos los primeros 20 tokens de cada método
print("\nTokens con SpaCy:", tokens_spacy[:20])
print("\nTokens con NLTK:", tokens_nltk[:20])
print("\nTokens con Stanza:", tokens_stanza[:20])
print("\nTokens con BERT:", tokens_bert[:20])


Tokens con SpaCy: ['investigación', 'sobre', 'el', 'sesgo', 'de', 'magnitud', '\n', 'tldr', '\n', 'los', 'sesgos', 'de', 'magnitud', 'en', 'estudios', 'sociales', 'se', 'refieren', 'a', 'errores']

Tokens con NLTK: ['investigación', 'sobre', 'el', 'sesgo', 'de', 'magnitud', 'tldr', 'los', 'sesgos', 'de', 'magnitud', 'en', 'estudios', 'sociales', 'se', 'refieren', 'a', 'errores', 'sistemáticos', 'que']

Tokens con Stanza: ['investigación', 'sobre', 'el', 'sesgo', 'de', 'magnitud', 'tldr', 'los', 'sesgos', 'de', 'magnitud', 'en', 'estudios', 'sociales', 'se', 'refieren', 'a', 'errores', 'sistemáticos', 'que']

Tokens con BERT: ['investigación', 'sobre', 'el', 'ses', '##go', 'de', 'magnitud', 't', '##ld', '##r', 'los', 'ses', '##gos', 'de', 'magnitud', 'en', 'estudios', 'sociales', 'se', 'refieren']


In [14]:
tokens_bert = tokenizer_bert.tokenize(texto_normalizado)

# Dividimos en fragmentos de 512 tokens
max_length = 512
chunks = [tokens_bert[i:i+max_length] for i in range(0, len(tokens_bert), max_length)]

print(f"Se generaron {len(chunks)} fragmentos de 512 tokens")

Se generaron 10 fragmentos de 512 tokens


In [15]:
def tokenizar_bert(texto):
    return tokenizer_bert.tokenize(texto)

In [16]:
print("\nTokens con BERT:", tokens_bert[:30])


Tokens con BERT: ['investigación', 'sobre', 'el', 'ses', '##go', 'de', 'magnitud', 't', '##ld', '##r', 'los', 'ses', '##gos', 'de', 'magnitud', 'en', 'estudios', 'sociales', 'se', 'refieren', 'a', 'errores', 'sistemático', '##s', 'que', 'afectan', 'la', 'estimación', 'cuantita', '##tiva']


In [17]:
# Eliminar stop words usando SpaCy
def eliminar_stopwords(tokens, nlp):
    return [token for token in tokens if token not in nlp.Defaults.stop_words]

tokens_sin_stopwords = eliminar_stopwords(tokens_spacy, nlp_spacy)
print("\nTokens sin stop words (primeros 20):", tokens_sin_stopwords[:20])


Tokens sin stop words (primeros 20): ['investigación', 'sesgo', 'magnitud', '\n', 'tldr', '\n', 'sesgos', 'magnitud', 'estudios', 'sociales', 'refieren', 'errores', 'sistemáticos', '\n', 'afectan', 'estimación', 'cuantitativa', 'efectos', ',', 'relaciones']


In [18]:
# Buscar números en el texto
numeros = re.findall(r'\d+', texto_minusculas)
print("Números encontrados en el texto:", numeros)

Números encontrados en el texto: ['7', '18', '18', '4', '26', '43', '24', '6', '10', '40', '10', '3', '1', '23', '36', '7', '20', '19', '48', '29', '21', '7', '40', '9', '18', '57', '21', '5', '46', '32', '43', '6', '1', '7', '18', '18', '25', '9', '20', '20', '21', '21', '18', '19', '19', '9', '23', '56', '48', '2', '5', '1', '2020', '11', '1', '6377', '2', '2022', '31', '43', '3', '2020', '15', '6', '0233625', '4', '2010', '5', '2013', '37', '2', '69', '88', '6', '2008', '37', '1', '3', '30', '7', '2012', '63', '539', '569', '8', '2012', '63', '539', '569', '9', '2022', '46', '3', '153', '166', '10', '2014', '345', '6203', '1502', '1505', '11', '1976', '24', '6', '582', '583', '12', '2012', '16', '2025', '345', '362', '17', '2023', '102', '11', '1196', '1204', '18', '2014', '43', '6', '1969', '1985', '19', '2021', '190', '8', '1604', '1612', '20', '2023', '7', '2', '1', '18', '21', '2021', '50', '5', '1708', '1730', '22', '2012', '23', '2020', '2020', '1', '13', '24', '2007', '157', 

In [19]:
for idx, numero in enumerate(numeros, start=1):
    print(f"Número {idx}: {numero}")

Número 1: 7
Número 2: 18
Número 3: 18
Número 4: 4
Número 5: 26
Número 6: 43
Número 7: 24
Número 8: 6
Número 9: 10
Número 10: 40
Número 11: 10
Número 12: 3
Número 13: 1
Número 14: 23
Número 15: 36
Número 16: 7
Número 17: 20
Número 18: 19
Número 19: 48
Número 20: 29
Número 21: 21
Número 22: 7
Número 23: 40
Número 24: 9
Número 25: 18
Número 26: 57
Número 27: 21
Número 28: 5
Número 29: 46
Número 30: 32
Número 31: 43
Número 32: 6
Número 33: 1
Número 34: 7
Número 35: 18
Número 36: 18
Número 37: 25
Número 38: 9
Número 39: 20
Número 40: 20
Número 41: 21
Número 42: 21
Número 43: 18
Número 44: 19
Número 45: 19
Número 46: 9
Número 47: 23
Número 48: 56
Número 49: 48
Número 50: 2
Número 51: 5
Número 52: 1
Número 53: 2020
Número 54: 11
Número 55: 1
Número 56: 6377
Número 57: 2
Número 58: 2022
Número 59: 31
Número 60: 43
Número 61: 3
Número 62: 2020
Número 63: 15
Número 64: 6
Número 65: 0233625
Número 66: 4
Número 67: 2010
Número 68: 5
Número 69: 2013
Número 70: 37
Número 71: 2
Número 72: 69
Número 7

In [20]:
# Importamos la librería 're', que es el módulo de expresiones regulares de Python.
# No necesita instalación porque viene incluida con el lenguaje.
import re

# Definimos la función que realizará la tokenización.
# Recibe una cadena de texto como entrada.
def tokenizar_con_regex_y_numeros(texto_minusculas):
    """
    Esta función tokeniza un texto utilizando una expresión regular
    diseñada para separar palabras, números y signos de puntuación.
    """

    # Definimos el patrón de la expresión regular. Se lee como una serie de "O" lógicos (|).
    # r'' indica que es una "raw string" para que los caracteres como '\' no se interpreten como secuencias de escape.
    # 1. \d+      : Busca una o más ocurrencias consecutivas de dígitos (números). Esto captura '500', '2025', '10'.
    # 2. |         : Actúa como un "O".
    # 3. [A-Za-zÁ-ú]+ : Busca una o más ocurrencias de letras, tanto mayúsculas como minúsculas.
    #                  Incluimos 'Á-ú' para capturar caracteres del español como 'á', 'é', 'í', 'ó', 'ú', 'ñ'.
    # 4. |         : Otro "O".
    # 5. [^\w\s]  : Busca cualquier caracter individual que NO sea (\^) un caracter de palabra (\w, que incluye letras, números y _)
    #                  Y que NO sea (\^) un espacio en blanco (\s). Esto nos permite capturar signos de puntuación como '.', ':', '$'.
    patron = r'\d+|[A-Za-zÁ-ú]+|[^\w\s]'

    # Utilizamos la función re.findall() para encontrar todas las subcadenas en el texto
    # que coincidan con nuestro patrón. Devuelve una lista con todos los hallazgos.
    tokens = re.findall(patron, texto_minusculas)

    # La función devuelve la lista de tokens encontrados.
    return tokens

In [21]:
# --- Proceso de ejecución ---

# 1. Definimos un texto de ejemplo que contiene palabras, números y puntuación.
# texto_ejemplo = "En 2025, el producto A-42 costará $500.50 y la entrega será a las 10:30."

# 2. Llamamos a nuestra función personalizada para tokenizar el texto.
tokens_resultado = tokenizar_con_regex_y_numeros(texto_minusculas)

# 3. Imprimimos el resultado para verificar cómo se separaron los elementos.
print(f"Texto original:\n{texto_minusculas}\n")
print(f"Tokens generados ({len(tokens_resultado)} tokens):\n{tokens_resultado}")

Texto original:
investigación sobre el sesgo de magnitud 
tldr 
los sesgos de magnitud en estudios sociales se refieren a errores sistemáticos que 
afectan la estimación cuantitativa de efectos, relaciones o parámetros en la 
investigación social, distorsionando tanto la dirección como la magnitud de los 
hallazgos y comprometiendo la validez de las conclusiones. 
introducción 
los sesgos de magnitud representan uno de los problemas metodológicos más 
críticos en la investigación social cuantitativa. estos sesgos se manifiestan como 
errores sistemáticos que afectan no solo la dirección de los efectos observados, sino 
también su magnitud, creando distorsiones significativas en la interpretación de los 
resultados [7], [18]. a diferencia de los errores aleatorios, los sesgos de magnitud 
introducen patrones consistentes de desviación que pueden llevar a conclusiones 
erróneas sobre la importancia práctica y estadística de los fenómenos sociales 
estudiados. 
definición y conceptualizac

In [22]:
!pip install spacy
!python -m spacy download es_core_news_sm
!pip install pdfplumber --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 123.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [23]:
import pdfplumber  # Para extraer texto de PDFs
import re  # Para trabajar con expresiones regulares
import spacy  # Para procesamiento de lenguaje natural (NLP)
import unicodedata  # Para normalizar caracteres Unicode

In [24]:
nlp = spacy.load("es_core_news_sm")  #cargar el modelo en español

In [25]:
# Importa la librería fitz, que es el nombre del módulo PyMuPDF
import fitz

# Usa la misma ruta de tu archivo
pdf_path = "/content/drive/MyDrive/Maestria en analítica estratégica de datos/4. Semestrre/Analisis_textos/Taller_2/Sesgos de magnitud.pdf"
texto_pdf = ""

try:
    # Abre el documento PDF con fitz
    with fitz.open(pdf_path) as doc:
        # Itera sobre cada página del documento
        for pagina in doc:
            # Extrae el texto de la página y lo añade a la variable
            texto_pdf += pagina.get_text()

    # Imprime el texto extraído (o una parte para verificar)
    print("¡Texto extraído con éxito!")
    print(texto_pdf[:500]) # Imprime los primeros 500 caracteres

except Exception as e:
    # Si ocurre un error, imprímelo para saber qué pasó
    print(f"No se pudo leer el PDF. El archivo podría estar dañado o no ser un PDF válido.")
    print(f"Error original: {e}")

¡Texto extraído con éxito!
Investigación sobre el sesgo de Magnitud 
TLDR 
Los sesgos de magnitud en estudios sociales se refieren a errores sistemáticos que 
afectan la estimación cuantitativa de efectos, relaciones o parámetros en la 
investigación social, distorsionando tanto la dirección como la magnitud de los 
hallazgos y comprometiendo la validez de las conclusiones. 
Introducción 
Los sesgos de magnitud representan uno de los problemas metodológicos más 
críticos en la investigación social cuantitativa. Estos sesg


In [26]:
# Extraer texto del PDF
with pdfplumber.open(pdf_path) as pdf:
    texto_pdf = ""
    for pagina in pdf.pages:
        texto_pdf += pagina.extract_text()

In [27]:
header_pattern = r"Investigación sobre el sesgo de Magnitud "

In [28]:
def extract_text_without_header(pdf_path):
    extracted_text = []  # Lista donde almacenaremos el texto de cada página

    with pdfplumber.open(pdf_path) as pdf:  # Abrir el archivo PDF
        for page in pdf.pages:  # Iterar por cada página del PDF
            text = page.extract_text()  # Extraer texto de la página
            if text:  # Verificar si hay texto en la página
                text = re.sub(header_pattern, "", text, flags=re.DOTALL)  # Eliminar encabezado con regex
                extracted_text.append(text)  # Guardar el texto limpio en la lista

    return " ".join(extracted_text)  # Unir todas las páginas en un solo texto

In [29]:
def preprocess_text(text):
    text = unicodedata.normalize("NFKD", text)  # Normalizar caracteres Unicode (ej. á → a)

    doc = nlp(text.lower())  # Convertir todo el texto a minúsculas y procesarlo con spaCy

    tokens = [token.text for token in doc if token.is_alpha and not token.is_stop]
    return tokens

In [30]:
raw_text = extract_text_without_header(pdf_path)  # Extraer texto limpio del PDF
tokens = preprocess_text(raw_text)  # Preprocesar el texto con spaCy

print(tokens[:50])  # Mostrar los primeros 50 tokens como prueba

['sesgo', 'magnitud', 'tldr', 'sesgos', 'magnitud', 'estudios', 'sociales', 'refieren', 'errores', 'afectan', 'cuantitativa', 'efectos', 'relaciones', 'social', 'distorsionando', 'magnitud', 'hallazgos', 'comprometiendo', 'validez', 'conclusiones', 'sesgos', 'magnitud', 'representan', 'problemas', 'social', 'cuantitativa', 'sesgos', 'manifiestan', 'errores', 'afectan', 'efectos', 'observados', 'magnitud', 'creando', 'distorsiones', 'significativas', 'resultados', 'diferencia', 'errores', 'aleatorios', 'sesgos', 'magnitud', 'introducen', 'patrones', 'consistentes', 'conclusiones', 'importancia', 'sociales', 'estudiados', 'completa']


In [31]:
with open("/content/drive/MyDrive/Maestria en analítica estratégica de datos/4. Semestrre/Analisis_textos/Taller_2/TALLER_2.txt", "w", encoding="utf-8") as f:
    f.write(" ".join(tokens))  # Guardar los tokens en un archivo de texto
print("Tokens guardados en 'TALLER_2.txt'")

Tokens guardados en 'TALLER_2.txt'
